# ANN Session 2 - Tensorflow

In [ ]:
!pip install tensorflow==2.0.0-beta1 

In [ ]:
!pip install matplotlib

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import matplotlib
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Numpy Introduction


## Reference


- Numpy Manual: https://docs.scipy.org/doc/numpy/

## Create Arrays

### Manual Creation

In [ ]:
a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
a1

### Reshape

In [ ]:
a2 = np.arange(1,10).reshape(3,3)
a2

### zeors, ones, empty, random, randn

In [ ]:
a3 = np.empty((3,3))
a4 = np.zeros((3,3))
a5 = np.ones((3,3))
a6 = np.random.random((3,3))
a7 = np.random.randn(3,3)

In [ ]:
print(a3)
print(a4)
print(a5)
print(a6)
print(a7)

## Array Propertites

### ndim, shape, size, dtype

In [ ]:
a1 = np.arange(1,10).reshape(3,3)
a1

In [ ]:
print(a1.ndim)
print(a1.shape)
print(a1.size)
print(a1.dtype)

### Change dtype

In [ ]:
a2 = a1.astype(float)
print(a1)
print(a1.dtype)
print(a2)
print(a2.dtype)

## Functions

### Basic

In [ ]:
a1 = np.array([[1,2,3], [4,5,6], [7,8,9]])
a2 = np.ones((3,3))
print(a1)
print(a2)

In [ ]:
print(a1 + a2)

In [ ]:
print(a1 - a2)

In [ ]:
print(a1 + 2 * a2)

In [ ]:
print(a1.sum())
print(a1.max())
print(a1.min())

### Customized Functions

In [ ]:
def gaussian(x, m, s):
    return (1.0 / (s * np.sqrt(2 * np.pi))) * np.exp(-np.power(x - m, 2.0) / (2 * np.power(s, 2.0)))

In [ ]:
x = np.linspace(-5, 5, 200)

In [ ]:
y = gaussian(x,0,1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x,y)
plt.show

# Tensorflow Basic

- Architecture: https://www.tensorflow.org/guide/extend/architecture

## Low Level API

- TensorFlow Core
- The interal exposed API for advanced users
- Not recommended for most users

## High Level API

- Keras: https://keras.io/
- Fast prototyping
- Production
- User friendly

## Demo 1 - Regression

**Regression Target Function**: y = x\*\*2 + y\*\*2

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)

X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

### Build the Model

- Input: [x, y]
- Ouput: z
- Num. of layers: 1 x Input + 1 x Hidden + 1 x Output = 3

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(64, activation='sigmoid', input_shape=(2,)),
        layers.Dense(1)
    ]
)
optimizer = keras.optimizers.RMSprop(0.01)
model.compile(
    loss='mse',
    optimizer=optimizer,
    metrics=['mse']
)
model.summary()

In [ ]:
def sampling(x, y):
    X, Y = np.meshgrid(x, y)
    Z = X**2 + Y**2
    samples = np.stack((X.flatten(),Y.flatten(),Z.flatten()), axis=1)
    np.random.shuffle(samples)
    return samples

def split_labels(samples):
    return (samples[:,0:-1], samples[:,-1])

def split_samples(samples, ratio):
    sep = int(samples.shape[0] * ratio)
    return (samples[0:sep], samples[sep:])

In [ ]:
x = np.linspace(-10, 10, 1000)
y = np.linspace(-10, 10, 1000)

samples = sampling(x, y)

train_samples, test_samples = split_samples(samples, 0.8)

(train_x, train_y) = split_labels(train_samples)
(test_x, test_y) = split_labels(test_samples)

model.fit(train_x, train_y, batch_size=100, epochs=10)

In [ ]:
ret = model.evaluate(test_x, test_y)

In [ ]:
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)

X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

v_samples = sampling(x, y)
v_x, v_actual = split_labels(v_samples)
v_y = model.predict(v_x)
model.evaluate(v_x, v_actual)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)
ax.plot_surface(X, Y, v_y.reshape(100,100))

## Demo 2 - Binary Classification

### Load Data - Titanic

In [ ]:
import pandas as pd
data_raw = pd.read_csv('data/titanic_train.csv')
data = pd.read_csv('data/titanic_clean_numeric.csv')

In [ ]:
data_raw.head()

In [ ]:
data.head()

### Split Data

In [ ]:
data_train = data.loc[:599]
data_val = data.loc[599:]

In [ ]:
train_x = data_train.loc[:, data.columns != "Survived"]
train_y = data_train.loc[:, data.columns == "Survived"]
val_x = data_val.loc[:, data.columns != "Survived"]
val_y = data_val.loc[:, data.columns == "Survived"]

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

### Build the Model

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(6, activation='sigmoid', input_shape=(11,)),
        layers.Dense(6, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ]
)
optimizer = keras.optimizers.Adam(0.01)
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['acc']
)
model.summary()

In [ ]:
history = model.fit(train_x, train_y, batch_size=10, epochs=100)

In [ ]:
ret = model.evaluate(val_x, val_y)

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['acc'], label="Accuracy")
ax.plot(history.history['loss'], label="LOSS")
ax.legend()